# Jira Integration Test Notebook

This notebook tests the Jira integration functionality including:
- Authentication
- Project operations
- Ticket management
- JQL search operations
- Fix version filtering
- Connection validation

In [ ]:
# Setup and imports
import os
import asyncio
import json
from typing import Dict, Any, List
from IPython.display import display, HTML

# Change to backend directory
os.chdir("./")
print(f"Current working directory: {os.getcwd()}")

In [ ]:
# Import Jira integration components
from app.integrations.factory import get_api_factory
from app.integrations.base.jira_interface import JiraTicket
from app.core.config import get_settings

print("Jira integration components imported successfully")

In [ ]:
# Initialize API factory and get Jira client
factory = get_api_factory()
jira_client = factory.create_client("jira", use_mock=True)  # Use mock for testing

print(f"Jira client created: {type(jira_client).__name__}")
print(f"Using mock implementation: {hasattr(jira_client, '_mock_data')}")

## 1. Authentication Test

In [ ]:
# Test authentication
async def test_authentication():
    try:
        auth_result = await jira_client.authenticate()
        print(f"Authentication result: {auth_result}")
        return auth_result
    except Exception as e:
        print(f"Authentication error: {e}")
        return False

# Run authentication test
auth_success = await test_authentication()
print(f"✅ Authentication test {'PASSED' if auth_success else 'FAILED'}")

## 2. Connection Validation Test

In [ ]:
# Test connection validation
async def test_connection_validation():
    try:
        connection_info = await jira_client.validate_connection()
        print("Connection validation result:")
        print(json.dumps(connection_info, indent=2))
        return connection_info
    except Exception as e:
        print(f"Connection validation error: {e}")
        return None

# Run connection validation test
connection_info = await test_connection_validation()
print(f"✅ Connection validation test {'PASSED' if connection_info else 'FAILED'}")

## 3. Project Operations Test

In [ ]:
# Test project operations
async def test_project_operations():
    try:
        # Get all projects
        projects = await jira_client.get_projects()
        print(f"Found {len(projects)} projects")
        
        for project in projects[:5]:  # Show first 5 projects
            print(f"  - {project.get('key', 'N/A')}: {project.get('name', 'N/A')}")
        
        if len(projects) > 5:
            print(f"  ... and {len(projects) - 5} more projects")
        
        return True
    except Exception as e:
        print(f"Project operations error: {e}")
        return False

# Run project operations test
project_test_success = await test_project_operations()
print(f"✅ Project operations test {'PASSED' if project_test_success else 'FAILED'}")

## 4. Ticket Operations Test

In [ ]:
# Test ticket operations
async def test_ticket_operations():
    test_ticket_key = "TEST-123"
    
    try:
        # Get specific ticket
        ticket = await jira_client.get_ticket(test_ticket_key)
        if ticket:
            print(f"Found ticket: {ticket.key} - {ticket.summary}")
            print(f"  Status: {ticket.status}")
            print(f"  Type: {ticket.issue_type}")
            print(f"  Assignee: {ticket.assignee}")
            print(f"  Fix Version: {ticket.fix_version}")
        else:
            print(f"Ticket {test_ticket_key} not found")
        
        return True
    except Exception as e:
        print(f"Ticket operations error: {e}")
        return False

# Run ticket operations test
ticket_test_success = await test_ticket_operations()
print(f"✅ Ticket operations test {'PASSED' if ticket_test_success else 'FAILED'}")

## 5. Fix Version Filtering Test

In [ ]:
# Test fix version filtering
async def test_fix_version_filtering():
    test_fix_version = "v1.0.0"
    test_project_keys = ["TEST", "DEMO"]
    
    try:
        # Get tickets by fix version
        tickets = await jira_client.get_tickets_by_fix_version(test_fix_version)
        print(f"Found {len(tickets)} tickets with fix version '{test_fix_version}'")
        
        for ticket in tickets[:5]:  # Show first 5 tickets
            print(f"  - {ticket.key}: {ticket.summary} (Status: {ticket.status})")
        
        if len(tickets) > 5:
            print(f"  ... and {len(tickets) - 5} more tickets")
        
        # Test with specific project keys
        project_tickets = await jira_client.get_tickets_by_fix_version(
            test_fix_version, test_project_keys
        )
        print(f"\nFound {len(project_tickets)} tickets in specified projects")
        
        return True
    except Exception as e:
        print(f"Fix version filtering error: {e}")
        return False

# Run fix version filtering test
fix_version_test_success = await test_fix_version_filtering()
print(f"✅ Fix version filtering test {'PASSED' if fix_version_test_success else 'FAILED'}")

## 6. JQL Search Test

In [ ]:
# Test JQL search operations
async def test_jql_search():
    test_jql_queries = [
        "project = TEST AND status = 'In Progress'",
        "fixVersion = 'v1.0.0' AND assignee is not EMPTY",
        "created >= '2024-01-01' AND priority in (High, Critical)",
        "summary ~ 'bug' OR description ~ 'error'",
    ]
    
    try:
        for i, jql in enumerate(test_jql_queries, 1):
            print(f"\n--- JQL Query {i} ---")
            print(f"Query: {jql}")
            
            tickets = await jira_client.search_tickets(jql, max_results=10)
            print(f"Found {len(tickets)} tickets")
            
            for ticket in tickets[:3]:  # Show first 3 results
                print(f"  - {ticket.key}: {ticket.summary}")
            
            if len(tickets) > 3:
                print(f"  ... and {len(tickets) - 3} more tickets")
        
        return True
    except Exception as e:
        print(f"JQL search error: {e}")
        return False

# Run JQL search test
jql_test_success = await test_jql_search()
print(f"✅ JQL search test {'PASSED' if jql_test_success else 'FAILED'}")

## 7. Advanced Search Scenarios

In [ ]:
# Test advanced search scenarios
async def test_advanced_search_scenarios():
    scenarios = [
        {
            "name": "Release Candidate Tickets",
            "jql": "fixVersion = 'v1.0.0-RC' AND status in ('Ready for Testing', 'In Testing')"
        },
        {
            "name": "High Priority Bugs",
            "jql": "issueType = Bug AND priority in (High, Critical) AND status != Closed"
        },
        {
            "name": "Unassigned Tasks",
            "jql": "assignee is EMPTY AND status != Closed"
        },
        {
            "name": "Recently Updated",
            "jql": "updated >= '-7d' ORDER BY updated DESC"
        }
    ]
    
    try:
        for scenario in scenarios:
            print(f"\n--- {scenario['name']} ---")
            tickets = await jira_client.search_tickets(scenario['jql'], max_results=5)
            print(f"Found {len(tickets)} tickets")
            
            for ticket in tickets:
                print(f"  - {ticket.key}: {ticket.summary} ({ticket.status})")
        
        return True
    except Exception as e:
        print(f"Advanced search scenarios error: {e}")
        return False

# Run advanced search scenarios test
advanced_search_success = await test_advanced_search_scenarios()
print(f"✅ Advanced search scenarios test {'PASSED' if advanced_search_success else 'FAILED'}")

## 8. Real vs Mock Comparison Test

In [ ]:
# Compare mock vs real implementations
async def compare_mock_vs_real():
    print("=== Mock Implementation ===")
    mock_client = factory.create_client("jira", use_mock=True)
    mock_auth = await mock_client.authenticate()
    print(f"Mock auth: {mock_auth}")
    
    print("\n=== Real Implementation ===")
    try:
        real_client = factory.create_client("jira", use_mock=False)
        real_auth = await real_client.authenticate()
        print(f"Real auth: {real_auth}")
    except Exception as e:
        print(f"Real implementation error (expected if no real credentials): {e}")

# Run comparison test
await compare_mock_vs_real()

## 9. Error Handling Test

In [ ]:
# Test error handling
async def test_error_handling():
    print("Testing error handling scenarios...")
    
    # Test with invalid ticket key
    try:
        await jira_client.get_ticket("INVALID-999")
    except Exception as e:
        print(f"✅ Expected error for invalid ticket: {type(e).__name__}")
    
    # Test with invalid JQL
    try:
        await jira_client.search_tickets("INVALID JQL SYNTAX", max_results=5)
    except Exception as e:
        print(f"✅ Expected error for invalid JQL: {type(e).__name__}")
    
    # Test with non-existent fix version
    try:
        await jira_client.get_tickets_by_fix_version("non-existent-version")
    except Exception as e:
        print(f"✅ Expected error for non-existent fix version: {type(e).__name__}")
    
    print("Error handling tests completed")

In [ ]:
# Run error handling test
await test_error_handling()

## 10. Performance Test

In [ ]:
# Test performance with multiple concurrent requests
async def test_performance():
    import time
    
    print("Testing performance with concurrent requests...")
    
    # Test single request timing
    start_time = time.time()
    await jira_client.get_projects()
    single_request_time = time.time() - start_time
    print(f"Single request time: {single_request_time:.3f}s")
    
    # Test multiple concurrent requests
    start_time = time.time()
    tasks = [
        jira_client.get_projects(),
        jira_client.get_tickets_by_fix_version("v1.0.0"),
        jira_client.search_tickets("project = TEST", max_results=10),
        jira_client.validate_connection()
    ]
    
    results = await asyncio.gather(*tasks, return_exceptions=True)
    concurrent_time = time.time() - start_time
    
    print(f"Concurrent requests time: {concurrent_time:.3f}s")
    print(f"Speedup: {single_request_time * len(tasks) / concurrent_time:.2f}x")
    
    # Check results
    successful_requests = sum(1 for r in results if not isinstance(r, Exception))
    print(f"Successful concurrent requests: {successful_requests}/{len(tasks)}")

# Run performance test
await test_performance()

## 11. Summary and Results

In [ ]:
# Display test summary
def display_test_summary():
    summary = {
        "Authentication": auth_success,
        "Connection Validation": bool(connection_info),
        "Project Operations": project_test_success,
        "Ticket Operations": ticket_test_success,
        "Fix Version Filtering": fix_version_test_success,
        "JQL Search": jql_test_success,
        "Advanced Search Scenarios": advanced_search_success,
    }
    
    print("\n" + "="*50)
    print("JIRA INTEGRATION TEST SUMMARY")
    print("="*50)
    
    for test_name, result in summary.items():
        status = "✅ PASS" if result else "❌ FAIL"
        print(f"{test_name:<25} {status}")
    
    total_tests = len(summary)
    passed_tests = sum(summary.values())
    print(f"\nTotal Tests: {total_tests}")
    print(f"Passed: {passed_tests}")
    print(f"Failed: {total_tests - passed_tests}")
    print(f"Success Rate: {(passed_tests/total_tests)*100:.1f}%")

# Display summary
display_test_summary()